# Artificial Intelligence Mini Project - AZ5411 

## AI Powered Alchemy - Poorna prakash S, Srilakshmi H and Thanya Gayathri N

## Review 1

## Reading the dataset

In [3]:
import pandas as pd
df1=pd.read_csv('DDH Data with Properties.csv')
df1.head()

,CID,SMILES,MolecularFormula,MolecularWeight,InChI,InChIKey,IUPACName,XLogP,ExactMass,MonoisotopicMass,...,FeatureAcceptorCount3D,FeatureDonorCount3D,FeatureAnionCount3D,FeatureCationCount3D,FeatureRingCount3D,FeatureHydrophobeCount3D,ConformerModelRMSD3D,EffectiveRotorCount3D,ConformerCount3D,pIC50
0,2744814,ClC1=CC(NC(=O)CSC2=NC=CC(=N2)C2=CSC(=N2)C2=CC=...,C21H14Cl2N4OS2,473.4,InChI=1S/C21H14Cl2N4OS2/c22-14-8-15(23)10-16(9...,LILOEJREEQFTPM-UHFFFAOYSA-N,"N-(3,5-dichlorophenyl)-2-[4-(2-phenyl-1,3-thia...",5.6,471.998609,471.998609,...,3.0,1.0,0.0,1.0,4.0,0.0,1.0,7.0,10.0,-0.477121255
1,2821293,CN1N=C(C=C1C(F)(F)F)C1=CC=C(S1)C1=CC=NC(SCC(=O...,C21H15ClF3N5OS2,510.0,"InChI=1S/C21H15ClF3N5OS2/c1-30-18(21(23,24)25)...",AWQBIBTZJKFLEW-UHFFFAOYSA-N,N-(4-chlorophenyl)-2-[4-[5-[1-methyl-5-(triflu...,4.9,509.035865,509.035865,...,3.0,1.0,0.0,1.0,4.0,0.0,1.2,8.0,10.0,-1
2,2820912,CSC1=C(C(C)=C(S1)C1=NC(C)=CS1)C1=CC=NC(SCC(=O)...,C22H19ClN4OS4,519.1,InChI=1S/C22H19ClN4OS4/c1-12-10-30-20(25-12)19...,WRXXISITJDZVCL-UHFFFAOYSA-N,N-(4-chlorophenyl)-2-[4-[4-methyl-2-methylsulf...,6.3,518.013024,518.013024,...,3.0,1.0,0.0,1.0,4.0,1.0,1.0,8.0,10.0,-1.041392685
3,2820914,CSC1=C(C(C)=C(S1)C1=NC(C)=CS1)C1=CC=NC(SCC(=O)...,C22H19ClN4OS4,519.1,InChI=1S/C22H19ClN4OS4/c1-12-10-30-20(25-12)19...,NNVVKOVHRSDRSQ-UHFFFAOYSA-N,N-(2-chlorophenyl)-2-[4-[4-methyl-2-methylsulf...,6.3,518.013024,518.013024,...,3.0,1.0,0.0,1.0,4.0,1.0,1.2,8.0,10.0,BLINDED
4,2744846,CC1=NC(=CS1)C1=NC(=CS1)C1=NC(SCC(=O)NC2=CC=C(C...,C19H14ClN5OS3,460.0,InChI=1S/C19H14ClN5OS3/c1-11-22-16(9-27-11)18-...,JEZYTEDGOJCVQS-UHFFFAOYSA-N,"N-(4-chlorophenyl)-2-[4-[2-(2-methyl-1,3-thiaz...",4.4,459.004901,459.004901,...,4.0,1.0,0.0,1.0,4.0,0.0,1.0,7.0,10.0,-1.146128036


## Check for missing values

In [5]:
df1.isnull().sum()

CID                         0
SMILES                      0
MolecularFormula            0
MolecularWeight             0
InChI                       0
InChIKey                    3
IUPACName                   3
XLogP                       3
ExactMass                   3
MonoisotopicMass            3
TPSA                        3
Complexity                  3
Charge                      3
HBondDonorCount             3
HBondAcceptorCount          3
RotatableBondCount          3
HeavyAtomCount              3
IsotopeAtomCount            3
AtomStereoCount             3
DefinedAtomStereoCount      3
UndefinedAtomStereoCount    3
BondStereoCount             3
DefinedBondStereoCount      3
UndefinedBondStereoCount    3
CovalentUnitCount           3
Volume3D                    3
XStericQuadrupole3D         4
YStericQuadrupole3D         4
ZStericQuadrupole3D         4
FeatureCount3D              4
FeatureAcceptorCount3D      4
FeatureDonorCount3D         4
FeatureAnionCount3D         4
FeatureCat

## Check for the shape

In [6]:
df1.shape

(104, 40)

## Variational Auto Encoder model

### 1. Adam Optimizer

In [30]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem

# Define a custom dataset class for SMILES strings
class SMILESDataset(Dataset):
    def __init__(self, smiles_list):
        self.smiles_list = smiles_list

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        return self.smiles_list[idx]

# Define the VAE architecture for SMILES
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, latent_size * 2)  # *2 for mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )
        self.latent_size = latent_size

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu_logvar = self.encoder(x)
        mu = mu_logvar[:, :self.latent_size]
        logvar = mu_logvar[:, self.latent_size:]
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Define training function
def train_vae(model, dataloader, optimizer, loss_function, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for smiles in dataloader:
            optimizer.zero_grad()
            inputs = smiles_to_fingerprint(smiles)
            inputs = inputs.to(device)
            recon_batch, mu, logvar = model(inputs)
            loss = loss_function(recon_batch, inputs, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader.dataset)}')

# Define loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Function to convert SMILES to Morgan fingerprint
def smiles_to_fingerprint(smiles_list, max_length=100):
    fingerprint_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
            fingerprint = np.array(fingerprint).astype(float)
            fingerprint_list.append(torch.from_numpy(fingerprint).float())
    padded_fingerprint = nn.utils.rnn.pad_sequence(fingerprint_list, batch_first=True, padding_value=0)
    return padded_fingerprint

# Example SMILES data
smiles_data = df1['SMILES']
# Define hyperparameters
input_size = 1024  # Size of input fingerprint vector
hidden_size = 256  # Size of hidden layer
latent_size = 64   # Size of latent space
batch_size = 32
learning_rate = 0.001
num_epochs = 100

# Create dataset and dataloader
dataset = SMILESDataset(smiles_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize VAE model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae_model = VAE(input_size, hidden_size, latent_size).to(device)

# Define optimizer with Adam
optimizer = optim.Adam(vae_model.parameters(), lr=learning_rate)

# Train the VAE
train_vae(vae_model, dataloader, optimizer, loss_function, num_epochs=num_epochs)

# Save the trained model weights
torch.save(vae_model.state_dict(), 'vae_model_adam.pth')


Epoch 1, Loss: 89.99439591627855
Epoch 2, Loss: 66.51373202984149
Epoch 3, Loss: 59.51921404325045
Epoch 4, Loss: 53.77498391958383
Epoch 5, Loss: 51.96206606351412
Epoch 6, Loss: 48.92057624230018
Epoch 7, Loss: 47.929218585674576
Epoch 8, Loss: 45.62708106407752
Epoch 9, Loss: 44.706348125751205
Epoch 10, Loss: 43.84571192814754
Epoch 11, Loss: 42.738507784329926
Epoch 12, Loss: 42.22827559251051
Epoch 13, Loss: 41.19544865534856
Epoch 14, Loss: 40.622418036827675
Epoch 15, Loss: 40.20584957416241
Epoch 16, Loss: 39.580386235163765
Epoch 17, Loss: 39.15222402719351
Epoch 18, Loss: 38.14267202524039
Epoch 19, Loss: 37.891137049748345
Epoch 20, Loss: 37.27158473088191
Epoch 21, Loss: 36.5541017972506
Epoch 22, Loss: 36.392272068904
Epoch 23, Loss: 35.606068244347206
Epoch 24, Loss: 35.44554959810697
Epoch 25, Loss: 34.318899594820465
Epoch 26, Loss: 34.40858840942383
Epoch 27, Loss: 34.3684076162485
Epoch 28, Loss: 33.923967801607574
Epoch 29, Loss: 33.63360830453726
Epoch 30, Loss: 34

### 2. Stochastic Gradient Descent

In [26]:
# Define the VAE architecture for SMILES
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, latent_size * 2)  # *2 for mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )
        self.latent_size = latent_size

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu_logvar = self.encoder(x)
        mu = mu_logvar[:, :self.latent_size]
        logvar = mu_logvar[:, self.latent_size:]
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Define training function
def train_vae(model, dataloader, optimizer, loss_function, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for smiles in dataloader:
            optimizer.zero_grad()
            inputs = smiles_to_fingerprint(smiles)
            inputs = inputs.to(device)
            recon_batch, mu, logvar = model(inputs)
            loss = loss_function(recon_batch, inputs, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader.dataset)}')

# Define loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Function to convert SMILES to Morgan fingerprint
def smiles_to_fingerprint(smiles_list, max_length=100):
    fingerprint_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
            fingerprint = np.array(fingerprint).astype(float)
            fingerprint_list.append(torch.from_numpy(fingerprint).float())
    padded_fingerprint = nn.utils.rnn.pad_sequence(fingerprint_list, batch_first=True, padding_value=0)
    return padded_fingerprint

# Example SMILES data
smiles_data = df1['SMILES']
# Define hyperparameters
input_size = 1024  # Size of input fingerprint vector
hidden_size = 256  # Size of hidden layer
latent_size = 64   # Size of latent space
batch_size = 32
learning_rate = 0.001
num_epochs = 5==0

# Create dataset and dataloader
dataset = SMILESDataset(smiles_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize VAE model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae_model = VAE(input_size, hidden_size, latent_size).to(device)

# Define optimizer with SGD
optimizer = optim.SGD(vae_model.parameters(), lr=learning_rate)

# Train the VAE
train_vae(vae_model, dataloader, optimizer, loss_function, num_epochs=num_epochs)

# Save the trained model weights
torch.save(vae_model.state_dict(), 'vae_model_sgd.pth')


Epoch 1, Loss: 73.73299584021935
Epoch 2, Loss: 50.53976499117338
Epoch 3, Loss: 46.898092416616585
Epoch 4, Loss: 43.96805631197416
Epoch 5, Loss: 42.690128326416016
Epoch 6, Loss: 41.14579567542443
Epoch 7, Loss: 40.138592353233925
Epoch 8, Loss: 39.99520580585186
Epoch 9, Loss: 39.422271728515625
Epoch 10, Loss: 38.44592461219201
Epoch 11, Loss: 37.68438192514273
Epoch 12, Loss: 37.981222299429085
Epoch 13, Loss: 36.83351575411283
Epoch 14, Loss: 36.64184218186598
Epoch 15, Loss: 36.16028565626878
Epoch 16, Loss: 36.030009049635666
Epoch 17, Loss: 35.89184981126051
Epoch 18, Loss: 34.63858883197491
Epoch 19, Loss: 34.60590274517353
Epoch 20, Loss: 34.38506874671349
Epoch 21, Loss: 33.463575509878304
Epoch 22, Loss: 33.31795853834886
Epoch 23, Loss: 32.2475826556866
Epoch 24, Loss: 33.00807101909931
Epoch 25, Loss: 32.40277231656588
Epoch 26, Loss: 31.601198343130257
Epoch 27, Loss: 31.721334604116585
Epoch 28, Loss: 32.08789297250601
Epoch 29, Loss: 31.97854995727539
Epoch 30, Loss:

### 3. Root Mean Squared Propagation

In [28]:
# Define the VAE architecture for SMILES
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, latent_size * 2)  # *2 for mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )
        self.latent_size = latent_size

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu_logvar = self.encoder(x)
        mu = mu_logvar[:, :self.latent_size]
        logvar = mu_logvar[:, self.latent_size:]
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Define training function
def train_vae(model, dataloader, optimizer, loss_function, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for smiles in dataloader:
            optimizer.zero_grad()
            inputs = smiles_to_fingerprint(smiles)
            inputs = inputs.to(device)
            recon_batch, mu, logvar = model(inputs)
            loss = loss_function(recon_batch, inputs, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader.dataset)}')

# Define loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.mse_loss(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Function to convert SMILES to Morgan fingerprint
def smiles_to_fingerprint(smiles_list, max_length=100):
    fingerprint_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
            fingerprint = np.array(fingerprint).astype(float)
            fingerprint_list.append(torch.from_numpy(fingerprint).float())
    padded_fingerprint = nn.utils.rnn.pad_sequence(fingerprint_list, batch_first=True, padding_value=0)
    return padded_fingerprint

# Example SMILES data
smiles_data = df1['SMILES']
# Define hyperparameters
input_size = 1024  # Size of input fingerprint vector
hidden_size = 256  # Size of hidden layer
latent_size = 64   # Size of latent space
batch_size = 32
learning_rate = 0.001
num_epochs = 100

# Create dataset and dataloader
dataset = SMILESDataset(smiles_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize VAE model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae_model = VAE(input_size, hidden_size, latent_size).to(device)

# Define optimizer with RMSprop
optimizer = optim.RMSprop(vae_model.parameters(), lr=learning_rate)

# Train the VAE
train_vae(vae_model, dataloader, optimizer, loss_function, num_epochs=num_epochs)

# Save the trained model weights
torch.save(vae_model.state_dict(), 'vae_model_rmsprop.pth')


Epoch 1, Loss: 104.00414012028621
Epoch 2, Loss: 52.071839552659256
Epoch 3, Loss: 47.26261285635141
Epoch 4, Loss: 45.93827643761268
Epoch 5, Loss: 44.47152944711539
Epoch 6, Loss: 44.01928124060998
Epoch 7, Loss: 42.67973239605244
Epoch 8, Loss: 41.3401853121244
Epoch 9, Loss: 40.63094681959886
Epoch 10, Loss: 40.0162972670335
Epoch 11, Loss: 38.904425400954025
Epoch 12, Loss: 38.09453993577223
Epoch 13, Loss: 37.53650635939378
Epoch 14, Loss: 36.67780421330379
Epoch 15, Loss: 36.05838834322416
Epoch 16, Loss: 35.88659902719351
Epoch 17, Loss: 35.04010537954477
Epoch 18, Loss: 34.92524983332707
Epoch 19, Loss: 34.164138500507065
Epoch 20, Loss: 33.96165422292856
Epoch 21, Loss: 33.98830824631911
Epoch 22, Loss: 34.408714294433594
Epoch 23, Loss: 33.626050802377556
Epoch 24, Loss: 32.84825926560622
Epoch 25, Loss: 33.16204100388747
Epoch 26, Loss: 33.17982248159555
Epoch 27, Loss: 32.69991713303786
Epoch 28, Loss: 32.66929582449106
Epoch 29, Loss: 32.236294966477615
Epoch 30, Loss: 32

## Saving the model

In [8]:
# Assuming vae_model is your trained VAE model
# Save only the model weights to a file
torch.save(vae_model.state_dict(), 'vae_model_weights.pth')

## Generating new molecules

In [9]:
import torch
vae_model.load_state_dict(torch.load('vae_model_weights.pth'))  # Load the trained model weights
vae_model.eval()  # Set the model to evaluation mode

# Define the number of molecules to generate
num_molecules = 20

# Generate new molecules
generated_molecules = []
with torch.no_grad():
    for _ in range(num_molecules):
        # Sample from the latent space (e.g., Gaussian distribution)
        latent_sample = torch.randn(1, vae_model.latent_size)

        # Decode the latent sample to generate a molecular structure
        decoded_molecule = vae_model.decoder(latent_sample)

        # Add the generated molecule to the list
        generated_molecules.append(decoded_molecule)

# Print the generated molecules
for idx, molecule in enumerate(generated_molecules):
    print(f"Molecule {idx + 1}: {molecule}")

Molecule 1: tensor([[ 0.0622, -0.0788,  0.0451,  ...,  0.1157, -0.0652, -0.0735]])
Molecule 2: tensor([[ 0.1505, -0.0474,  0.0889,  ..., -0.0147,  0.0341, -0.1232]])
Molecule 3: tensor([[-0.0211, -0.1066,  0.0700,  ...,  0.0662, -0.0384,  0.0293]])
Molecule 4: tensor([[-0.0493,  0.0044,  0.0548,  ...,  0.0685, -0.0520,  0.0157]])
Molecule 5: tensor([[-0.1191, -0.0528,  0.0397,  ...,  0.2404,  0.0379, -0.1637]])
Molecule 6: tensor([[-0.0167,  0.1652, -0.0038,  ..., -0.0151, -0.1328,  0.0911]])
Molecule 7: tensor([[-0.0239,  0.3416,  0.0817,  ...,  0.0559,  0.0324, -0.0798]])
Molecule 8: tensor([[-0.1857,  0.0886,  0.0935,  ..., -0.1338, -0.0724, -0.0186]])
Molecule 9: tensor([[ 0.0240,  0.2164,  0.1047,  ...,  0.0025,  0.1231, -0.0603]])
Molecule 10: tensor([[ 0.0570,  0.2507, -0.0237,  ..., -0.0241, -0.0310,  0.0646]])
Molecule 11: tensor([[-0.0891,  0.1776, -0.0454,  ...,  0.0529, -0.0412, -0.0484]])
Molecule 12: tensor([[ 0.0397,  0.0921,  0.2142,  ...,  0.1068,  0.0847, -0.1989]])
M

## Generative Adversarial Network

### 1. Adam Optimizer

In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem

# Define a custom dataset class for SMILES strings
class SMILESDataset(Dataset):
    def __init__(self, smiles_list):
        self.smiles_list = smiles_list

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        return self.smiles_list[idx]

# Define the Generator architecture for SMILES
class Generator(nn.Module):
    def __init__(self, latent_size, output_size):
        super(Generator, self).__init__()
        self.latent_size = latent_size
        self.output_size = output_size

        self.model = nn.Sequential(
            nn.Linear(latent_size, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_size)
        )

    def forward(self, z):
        return self.model(z)

# Define the Discriminator architecture for SMILES
class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.input_size = input_size

        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Define training function for GAN
def train_gan(generator, discriminator, dataloader, g_optimizer, d_optimizer, criterion, num_epochs=10):
    generator.train()
    discriminator.train()
    for epoch in range(num_epochs):
        g_running_loss = 0.0
        d_running_loss = 0.0
        for smiles in dataloader:
            real_smiles = smiles_to_tensor(smiles).to(device)
            
            # Train discriminator with real data
            d_optimizer.zero_grad()
            real_output = discriminator(real_smiles)
            real_labels = torch.ones(real_smiles.size(0), 1).to(device)
            d_real_loss = criterion(real_output, real_labels)
            
            # Train discriminator with fake data
            latent_samples = torch.randn(real_smiles.size(0), latent_size).to(device)
            fake_smiles = generator(latent_samples)
            fake_output = discriminator(fake_smiles.detach())
            fake_labels = torch.zeros(real_smiles.size(0), 1).to(device)
            d_fake_loss = criterion(fake_output, fake_labels)
            
            # Update discriminator weights
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()
            d_running_loss += d_loss.item()
            
            # Train generator
            g_optimizer.zero_grad()
            fake_output = discriminator(fake_smiles)
            g_loss = criterion(fake_output, real_labels)
            g_loss.backward()
            g_optimizer.step()
            g_running_loss += g_loss.item()
            
        print(f'Epoch {epoch+1}, Generator Loss: {g_running_loss / len(dataloader)}, Discriminator Loss: {d_running_loss / len(dataloader)}')

# Function to convert SMILES to tensor
def smiles_to_tensor(smiles_list, max_length=100):
    tensor_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
            fingerprint = np.array(fingerprint).astype(float)
            tensor_list.append(torch.from_numpy(fingerprint).float())
    padded_tensor = nn.utils.rnn.pad_sequence(tensor_list, batch_first=True, padding_value=0)
    return padded_tensor

# Example SMILES data
smiles_data = df1['SMILES']
latent_size = 100
output_size = 1024
batch_size = 64
learning_rate = 0.0002
num_epochs = 20

# Create dataset and dataloader
dataset = SMILESDataset(smiles_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize Generator and Discriminator
generator = Generator(latent_size, output_size).to(device)
discriminator = Discriminator(output_size).to(device)

# Define optimizers and criterion
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

# Train the GAN
train_gan(generator, discriminator, dataloader, g_optimizer, d_optimizer, criterion, num_epochs=num_epochs)

# Save the trained generator weights
torch.save(generator.state_dict(), 'generator_model.pth')

Epoch 1, Generator Loss: 0.6855122745037079, Discriminator Loss: 1.3769463300704956
Epoch 2, Generator Loss: 0.6848061978816986, Discriminator Loss: 1.3486742973327637
Epoch 3, Generator Loss: 0.683135986328125, Discriminator Loss: 1.3228106498718262
Epoch 4, Generator Loss: 0.679880291223526, Discriminator Loss: 1.2991365790367126
Epoch 5, Generator Loss: 0.6771744787693024, Discriminator Loss: 1.2706681489944458
Epoch 6, Generator Loss: 0.6709451973438263, Discriminator Loss: 1.2490885853767395
Epoch 7, Generator Loss: 0.6650155186653137, Discriminator Loss: 1.2289321422576904
Epoch 8, Generator Loss: 0.6576692163944244, Discriminator Loss: 1.209460437297821
Epoch 9, Generator Loss: 0.6532025337219238, Discriminator Loss: 1.194105625152588
Epoch 10, Generator Loss: 0.6499802768230438, Discriminator Loss: 1.1828950643539429
Epoch 11, Generator Loss: 0.644739955663681, Discriminator Loss: 1.1910693049430847
Epoch 12, Generator Loss: 0.6535230875015259, Discriminator Loss: 1.18376415967

### 2. Stochastic Gradient Descent

In [38]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem

# Define a custom dataset class for SMILES strings
class SMILESDataset(Dataset):
    def __init__(self, smiles_list):
        self.smiles_list = smiles_list

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        return self.smiles_list[idx]

# Define the Generator architecture for SMILES
class Generator(nn.Module):
    def __init__(self, latent_size, output_size):
        super(Generator, self).__init__()
        self.latent_size = latent_size
        self.output_size = output_size

        self.model = nn.Sequential(
            nn.Linear(latent_size, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_size)
        )

    def forward(self, z):
        return self.model(z)

# Define the Discriminator architecture for SMILES
class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.input_size = input_size

        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Define training function for GAN
def train_gan(generator, discriminator, dataloader, g_optimizer, d_optimizer, criterion, num_epochs=10):
    generator.train()
    discriminator.train()
    for epoch in range(num_epochs):
        g_running_loss = 0.0
        d_running_loss = 0.0
        for smiles in dataloader:
            real_smiles = smiles_to_tensor(smiles).to(device)
            
            # Train discriminator with real data
            d_optimizer.zero_grad()
            real_output = discriminator(real_smiles)
            real_labels = torch.ones(real_smiles.size(0), 1).to(device)
            d_real_loss = criterion(real_output, real_labels)
            
            # Train discriminator with fake data
            latent_samples = torch.randn(real_smiles.size(0), latent_size).to(device)
            fake_smiles = generator(latent_samples)
            fake_output = discriminator(fake_smiles.detach())
            fake_labels = torch.zeros(real_smiles.size(0), 1).to(device)
            d_fake_loss = criterion(fake_output, fake_labels)
            
            # Update discriminator weights
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()
            d_running_loss += d_loss.item()
            
            # Train generator
            g_optimizer.zero_grad()
            fake_output = discriminator(fake_smiles)
            g_loss = criterion(fake_output, real_labels)
            g_loss.backward()
            g_optimizer.step()
            g_running_loss += g_loss.item()
            
        print(f'Epoch {epoch+1}, Generator Loss: {g_running_loss / len(dataloader)}, Discriminator Loss: {d_running_loss / len(dataloader)}')

# Function to convert SMILES to tensor
def smiles_to_tensor(smiles_list, max_length=100):
    tensor_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
            fingerprint = np.array(fingerprint).astype(float)
            tensor_list.append(torch.from_numpy(fingerprint).float())
    padded_tensor = nn.utils.rnn.pad_sequence(tensor_list, batch_first=True, padding_value=0)
    return padded_tensor

# Example SMILES data
smiles_data = df1['SMILES']
latent_size = 100
output_size = 1024
batch_size = 64
learning_rate = 0.0002
num_epochs = 20

# Create dataset and dataloader
dataset = SMILESDataset(smiles_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize Generator and Discriminator
generator = Generator(latent_size, output_size).to(device)
discriminator = Discriminator(output_size).to(device)

# Define optimizers and criterion
g_optimizer = optim.SGD(generator.parameters(), lr=learning_rate)
d_optimizer = optim.SGD(discriminator.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

# Train the GAN
train_gan(generator, discriminator, dataloader, g_optimizer, d_optimizer, criterion, num_epochs=num_epochs)

# Save the trained generator weights
torch.save(generator.state_dict(), 'generator_model.pth')

Epoch 1, Generator Loss: 0.6812567114830017, Discriminator Loss: 1.3893267512321472
Epoch 2, Generator Loss: 0.6814540326595306, Discriminator Loss: 1.3888244032859802
Epoch 3, Generator Loss: 0.6809656620025635, Discriminator Loss: 1.3897905945777893
Epoch 4, Generator Loss: 0.6811710298061371, Discriminator Loss: 1.3892515897750854
Epoch 5, Generator Loss: 0.6810757219791412, Discriminator Loss: 1.389388084411621
Epoch 6, Generator Loss: 0.6817755699157715, Discriminator Loss: 1.3887900114059448
Epoch 7, Generator Loss: 0.68106409907341, Discriminator Loss: 1.3892813920974731
Epoch 8, Generator Loss: 0.6811458170413971, Discriminator Loss: 1.3890327215194702
Epoch 9, Generator Loss: 0.680977463722229, Discriminator Loss: 1.38931143283844
Epoch 10, Generator Loss: 0.6811736822128296, Discriminator Loss: 1.3889999985694885
Epoch 11, Generator Loss: 0.6814141571521759, Discriminator Loss: 1.3887261748313904
Epoch 12, Generator Loss: 0.6813799440860748, Discriminator Loss: 1.388796329498

### 3. Root Mean Squared Propagation

In [37]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem

# Define a custom dataset class for SMILES strings
class SMILESDataset(Dataset):
    def __init__(self, smiles_list):
        self.smiles_list = smiles_list

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        return self.smiles_list[idx]

# Define the Generator architecture for SMILES
class Generator(nn.Module):
    def __init__(self, latent_size, output_size):
        super(Generator, self).__init__()
        self.latent_size = latent_size
        self.output_size = output_size

        self.model = nn.Sequential(
            nn.Linear(latent_size, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, output_size)
        )

    def forward(self, z):
        return self.model(z)

# Define the Discriminator architecture for SMILES
class Discriminator(nn.Module):
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.input_size = input_size

        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Define training function for GAN
def train_gan(generator, discriminator, dataloader, g_optimizer, d_optimizer, criterion, num_epochs=10):
    generator.train()
    discriminator.train()
    for epoch in range(num_epochs):
        g_running_loss = 0.0
        d_running_loss = 0.0
        for smiles in dataloader:
            real_smiles = smiles_to_tensor(smiles).to(device)
            
            # Train discriminator with real data
            d_optimizer.zero_grad()
            real_output = discriminator(real_smiles)
            real_labels = torch.ones(real_smiles.size(0), 1).to(device)
            d_real_loss = criterion(real_output, real_labels)
            
            # Train discriminator with fake data
            latent_samples = torch.randn(real_smiles.size(0), latent_size).to(device)
            fake_smiles = generator(latent_samples)
            fake_output = discriminator(fake_smiles.detach())
            fake_labels = torch.zeros(real_smiles.size(0), 1).to(device)
            d_fake_loss = criterion(fake_output, fake_labels)
            
            # Update discriminator weights
            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()
            d_running_loss += d_loss.item()
            
            # Train generator
            g_optimizer.zero_grad()
            fake_output = discriminator(fake_smiles)
            g_loss = criterion(fake_output, real_labels)
            g_loss.backward()
            g_optimizer.step()
            g_running_loss += g_loss.item()
            
        print(f'Epoch {epoch+1}, Generator Loss: {g_running_loss / len(dataloader)}, Discriminator Loss: {d_running_loss / len(dataloader)}')

# Function to convert SMILES to tensor
def smiles_to_tensor(smiles_list, max_length=100):
    tensor_list = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
            fingerprint = np.array(fingerprint).astype(float)
            tensor_list.append(torch.from_numpy(fingerprint).float())
    padded_tensor = nn.utils.rnn.pad_sequence(tensor_list, batch_first=True, padding_value=0)
    return padded_tensor

# Example SMILES data
smiles_data = df1['SMILES']
latent_size = 100
output_size = 1024
batch_size = 64
learning_rate = 0.0002
num_epochs = 20

# Create dataset and dataloader
dataset = SMILESDataset(smiles_data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize Generator and Discriminator
generator = Generator(latent_size, output_size).to(device)
discriminator = Discriminator(output_size).to(device)

# Define optimizers and criterion
g_optimizer = optim.RMSprop(generator.parameters(), lr=learning_rate)
d_optimizer = optim.RMSprop(discriminator.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

# Train the GAN
train_gan(generator, discriminator, dataloader, g_optimizer, d_optimizer, criterion, num_epochs=num_epochs)

# Save the trained generator weights
torch.save(generator.state_dict(), 'generator_model.pth')


Epoch 1, Generator Loss: 0.7479954361915588, Discriminator Loss: 1.3523942828178406
Epoch 2, Generator Loss: 0.7173742353916168, Discriminator Loss: 1.1401679515838623
Epoch 3, Generator Loss: 0.6339037716388702, Discriminator Loss: 1.5157968997955322
Epoch 4, Generator Loss: 0.7592118978500366, Discriminator Loss: 1.4662600755691528
Epoch 5, Generator Loss: 0.7702058255672455, Discriminator Loss: 1.3066260814666748
Epoch 6, Generator Loss: 0.7477568089962006, Discriminator Loss: 1.332560956478119
Epoch 7, Generator Loss: 0.7205433249473572, Discriminator Loss: 1.3181023001670837
Epoch 8, Generator Loss: 0.739948034286499, Discriminator Loss: 1.293932318687439
Epoch 9, Generator Loss: 0.7708102166652679, Discriminator Loss: 1.2207165360450745
Epoch 10, Generator Loss: 0.7617858946323395, Discriminator Loss: 1.20681893825531
Epoch 11, Generator Loss: 0.7763881981372833, Discriminator Loss: 1.165480613708496
Epoch 12, Generator Loss: 0.8110237419605255, Discriminator Loss: 1.056904613971